<a href="https://colab.research.google.com/github/LinCheungS/Deep_Learning_TF2/blob/master/%E6%A8%A1%E5%9E%8B%E4%BF%9D%E5%AD%98%E4%B8%8E%E9%83%A8%E7%BD%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 模型保存与部署

![](https://raw.githubusercontent.com/LinCheungS/PicGo_Image_Storage/master/2020-1/20200602065813.svg)

In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

In [4]:
# 数据集 #1
from sklearn.preprocessing import StandardScaler
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
print(x_valid.shape, y_valid.shape)

(5000, 28, 28) (5000,)


## 保存加载keras模型

In [14]:
"""保存keras模型"""
keras.backend.clear_session()
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer = "sgd",
              metrics = ["accuracy"])

# 设置保存路径
logdir = './graph_def_and_weights'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,"fashion_mnist_weights.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,
                                    save_best_only = True,
                                    save_weights_only = False), # 设置是否只保存参数
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
]
history = model.fit(x_train_scaled, y_train, epochs=5,
                    validation_data=(x_valid_scaled, y_valid),
                    callbacks = callbacks)
model.evaluate(x_test_scaled, y_test)

Epoch 1/5
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5318 - accuracy: 0.8118 - val_loss: 0.4207 - val_accuracy: 0.8478
Epoch 2/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3883 - accuracy: 0.8604 - val_loss: 0.3721 - val_accuracy: 0.8680
Epoch 3/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3508 - accuracy: 0.8733 - val_loss: 0.3561 - val_accuracy: 0.8722
Epoch 4/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3262 - accuracy: 0.8828 - val_loss: 0.3351 - val_accuracy: 0.8740
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3623 - accuracy: 0.8668


[0.3623450994491577, 0.8668000102043152]

In [0]:
"""保存只有参数的keras模型"""
model.save_weights(os.path.join(logdir, "fashion_mnist_weights_2.h5"))

In [16]:
"""加载有网络结构的keras模型"""
loaded_model = keras.models.load_model(os.path.join(logdir, "fashion_mnist_weights.h5"))
loaded_model.evaluate(x_test_scaled, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3623 - accuracy: 0.8668


[0.3623450994491577, 0.8668000102043152]

In [17]:
"""加载只有参数的keras模型"""
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer = "sgd",
              metrics = ["accuracy"])

# 设置加载参数模型路径
logdir = './graph_def_and_weights'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, "fashion_mnist_weights_2.h5")
# 加载参数
model.load_weights(output_model_file)
# 衡量模型
model.evaluate(x_test_scaled, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3623 - accuracy: 0.8668


[0.3623450994491577, 0.8668000102043152]

## keras转换saved_model

In [18]:
# keras转换saved_model
tf.saved_model.save(model, "./keras_saved_graph")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./keras_saved_graph/assets


In [19]:
# 命令行打印save_model结构
!saved_model_cli show --dir ./keras_saved_graph --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_1_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28)
        name: serving_default_flatten_1_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_5'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0601 23:26:52.302783 140691341145984 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/

In [20]:
!saved_model_cli show --dir ./keras_saved_graph \
    --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_1_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28)
      name: serving_default_flatten_1_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_5'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [22]:
# 命令行做预测,  用上面打印的信息
!saved_model_cli run --dir ./keras_saved_graph --tag_set serve \
    --signature_def serving_default \
    --input_exprs 'flatten_1_input=np.ones((2, 28, 28))'

2020-06-01 23:29:23.013343: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-01 23:29:23.016033: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-06-01 23:29:23.016107: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (3f81888b9c22): /proc/driver/nvidia/version does not exist
2020-06-01 23:29:23.043489: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-06-01 23:29:23.044066: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55cd1593a840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-06-01 23:29:23.044131: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
W0601 23:29:23.051908 140065325549440 deprecation.py:323]

In [23]:
# python做预测
loaded_saved_model = tf.saved_model.load('./keras_saved_graph')
print(list(loaded_saved_model.signatures.keys()))

['serving_default']


In [24]:
inference = loaded_saved_model.signatures['serving_default']
print(inference)

In [25]:
print(inference.structured_outputs)

{'dense_5': TensorSpec(shape=(None, 10), dtype=tf.float32, name='dense_5')}


In [27]:
results = inference(tf.constant(x_test_scaled[0:1]))
print(results['dense_5'])

tf.Tensor(
[[7.1393069e-06 6.6101006e-06 7.0064234e-06 4.0687082e-06 7.8821331e-06
  1.8722540e-02 2.4624796e-05 2.8391231e-02 5.0716382e-04 9.5232177e-01]], shape=(1, 10), dtype=float32)


## 签名函数转save_model

In [28]:
# 生成带签名的函数
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z, 3)
# 将带签名的函数转成concreate function
cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None], tf.int32))

print(cube_func_int32)
print(cube(tf.constant([1, 2, 3])))

tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [29]:
to_export = tf.Module()
# 加一个成员变量cube
to_export.cube = cube
tf.saved_model.save(to_export, "./signature_to_saved_model")

INFO:tensorflow:Assets written to: ./signature_to_saved_model/assets


In [30]:
!saved_model_cli show --dir ./signature_to_saved_model --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['x'] tensor_info:
        dtype: DT_INT32
        shape: (-1)
        name: serving_default_x:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_INT32
        shape: (-1)
        name: PartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: 'cube'
    Option #1
      Callable with:
        Argument #1
          x: TensorSpec(shape=(None,), dtype=tf.int32, name=u'x')


In [31]:
imported = tf.saved_model.load('./signature_to_saved_model')
imported.cube(tf.constant([2]))

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([8], dtype=int32)>

## TensorflowJS

https://www.tensorflow.org/js/tutorials/conversion/import_keras

## Tensorflow安卓

## 读取TFLite

In [0]:
with open('/content/tflite_models/keras_tflite', 'rb') as f:
    concrete_func_tflite = f.read()
    
interpreter = tf.lite.Interpreter(model_content = concrete_func_tflite)
# 允许分配内存
interpreter.allocate_tensors()

In [48]:
# 获得输入和输出
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

[{'name': 'flatten_input', 'index': 0, 'shape': array([ 1, 28, 28], dtype=int32), 'shape_signature': array([ 1, 28, 28], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 12, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [49]:
# 生成输入数据
input_data = tf.constant(np.ones(input_details[0]['shape'], dtype=np.float32))
# 将输入数据塞给input_details[0]['index']
interpreter.set_tensor(input_details[0]['index'], input_data)
# 启动interpreter
interpreter.invoke()
# 获得输出
output_results = interpreter.get_tensor(output_details[0]['index'])
print(output_results)

[[0.19649196 0.04624215 0.06946057 0.05645237 0.04555378 0.02107765
  0.07102709 0.01042284 0.45968488 0.02358679]]


## keras转concrete_function

In [33]:
loaded_keras_model = keras.models.load_model(
    '/content/graph_def_and_weights/fashion_mnist_weights.h5')
loaded_keras_model(np.ones((1, 28, 28)))

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.19649196, 0.04624216, 0.06946056, 0.0564524 , 0.04555378,
        0.02107765, 0.0710271 , 0.01042284, 0.45968476, 0.0235868 ]],
      dtype=float32)>

In [0]:
run_model = tf.function(lambda x : loaded_keras_model(x))
keras_concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(loaded_keras_model.inputs[0].shape,
                  loaded_keras_model.inputs[0].dtype))

In [35]:
keras_concrete_func(tf.constant(np.ones((1, 28, 28), dtype=np.float32)))

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.19649196, 0.04624216, 0.06946056, 0.0564524 , 0.04555378,
        0.02107765, 0.0710271 , 0.01042284, 0.45968476, 0.0235868 ]],
      dtype=float32)>

## save_model转concrete_function

In [0]:
inference = loaded_saved_model.signatures['serving_default']

## TFLite量化

In [0]:
"""Keras模型量化"""
loaded_keras_model = keras.models.load_model('/content/graph_def_and_weights/fashion_mnist_weights.h5')
keras_to_tflite_converter = tf.lite.TFLiteConverter.from_keras_model(loaded_keras_model)
# 设置量化, 转成8位
keras_to_tflite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
keras_tflite = keras_to_tflite_converter.convert()
# 生成量化过的TFLite
if not os.path.exists('./tflite_models'):
    os.mkdir('./tflite_models')
with open('./tflite_models/quantized_keras_tflite', 'wb') as f:
    f.write(keras_tflite)

In [0]:
"""concreate function 量化"""
# kearas 转 concreate func
run_model = tf.function(lambda x : loaded_keras_model(x))
keras_concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(loaded_keras_model.inputs[0].shape,
                  loaded_keras_model.inputs[0].dtype))
keras_concrete_func(tf.constant(np.ones((1, 28, 28), dtype=np.float32)))
# 量化
concrete_func_to_tflite_converter = tf.lite.TFLiteConverter.from_concrete_functions([keras_concrete_func])
concrete_func_to_tflite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
concrete_func_tflite = concrete_func_to_tflite_converter.convert()
# 写入
with open('./tflite_models/quantized_concrete_func_tflite', 'wb') as f:
    f.write(concrete_func_tflite)

In [0]:
"""save_model量化"""
saved_model_to_tflite_converter = tf.lite.TFLiteConverter.from_saved_model('/content/keras_saved_graph')
saved_model_to_tflite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
saved_model_tflite = saved_model_to_tflite_converter.convert()

with open('./tflite_models/quantized_saved_model_tflite', 'wb') as f:
    f.write(saved_model_tflite)

## 各种模型转成TFLite

In [36]:
"""转换 SavedModel 格式模型"""

# 建立一个简单的模型。
root = tf.train.Checkpoint()
root.v1 = tf.Variable(3.)
root.v2 = tf.Variable(2.)
root.f = tf.function(lambda x: root.v1 * root.v2 * x)

# 保存模型。
export_dir = "/tmp/test_saved_model"
input_data = tf.constant(1., shape=[1, 1])
to_save = root.f.get_concrete_function(input_data)
tf.saved_model.save(root, export_dir, to_save)

# 转换模型。
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/test_saved_model/assets


In [0]:
"""转换 Keras 模型"""

# 创建一个简单的 Keras 模型。
x = [-1, 0, 1, 2, 3, 4]
y = [-3, -1, 1, 3, 5, 7]

model = tf.keras.models.load_model("/content/graph_def_and_weights/fashion_mnist_weights.h5")

# 转换模型。
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 写入模型
if not os.path.exists('./tflite_models'):
    os.mkdir('./tflite_models')
with open('./tflite_models/keras_tflite', 'wb') as f:
    f.write(tflite_model)

In [0]:
""""转换 concrete function """
# 以下示例展示了如何将 TensorFlow 中的 concrete function 转换为TensorFlow Lite 中的 FlatBuffer 格式。

# 建立一个模型。
root = tf.train.Checkpoint()
root.v1 = tf.Variable(3.)
root.v2 = tf.Variable(2.)
root.f = tf.function(lambda x: root.v1 * root.v2 * x)

# 生成 concrete function。
input_data = tf.constant(1., shape=[1, 1])
concrete_func = root.f.get_concrete_function(input_data)

# 转换模型。
#
# `from_concrete_function` 的传入参数被设计为一个一个 concrete function 的列表，然而
# 现阶段仅支持每次调用时仅接受一个concrete function。
# 同时转换多个concrete function的功能正在开发中。
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()